# Generate Cloud-Optimized Geotiffs from tiles


## Load Libraries

Check that gdal is installed

In [1]:
!gdalinfo --version

GDAL 3.6.3, released 2023/03/07


In [1]:
from osgeo import gdal
import subprocess
import json 
import pandas as pd
from google.cloud import storage
import os
import glob


In [2]:
os.environ['GS_NO_SIGN_REQUEST'] = 'YES'
os.environ['GDAL_NUM_THREADS'] = 'ALL_CPUS'

Log in to google cloud if needed 

In [4]:
#!{gcloud auth login --update-adc}

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=VyOXZ5V8gPMkYZP6mD9Z7YuzWnhidI&access_type=offline&code_challenge=iumn1jETg_zGLsaheAEjVofEZZnGFSReH6xDxgJ_SjU&code_challenge_method=S256


Application default credentials (ADC) were updated.

You are now logged in as [cnilsen@gmail.com].
Your current project is [swhm-dev].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [4]:
!{gcloud config set project swhm-dev}

Updated property [core/project].


## Functions

A function to list blobs on the storage bucket 

In [5]:
def list_blobs_with_prefix(bucket_name, prefix, delimiter=None):
    """Lists all the blobs in the bucket that begin with the prefix.
    """
    storage_client = storage.Client()
   
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=delimiter)

    # Note: The call returns a response only when the iterator is consumed.
    blob_list = []
    for blob in blobs:
        blob_list.append(blob.name)

    if delimiter:
        print("Prefixes:")
        for prefix in blobs.prefixes:
            blob_list.append([prefix])
    
    return blob_list


### 1. Download images

In [6]:
def dl(lay_name):
    cmd = f'gsutil -m cp -R gs://swhm-image-exports/{lay_name} .'
    !{cmd}

### 2. Reproject images

saves reprojected images to /tmp 

In [7]:
def reproject(lay_name, target_crs='EPSG:3857'): 

    directory_path = lay_name
               #make a list of the files in the directory 
    files = os.listdir(directory_path+"/reprojected")
    print(files)
    # create a new file for writing
    list_file = "files.txt"
    try:
        os.remove(list_file)
    except OSError:
        pass
    
    for filename in files:
        if filename.endswith(".tif"):
            input_path = os.path.join(directory_path, filename)
            output_path = os.path.join(directory_path+"/reprojected", filename)
            cmd = f'gdalwarp -t_srs {target_crs} -overwrite {input_path} {output_path}'
            !{cmd}

In [8]:
# cmd = f'gdalwarp -t_srs "EPSG:3857"-overwrite runoff_testing.tif testing_translate.tif'
# !{cmd}

In [9]:
# def write_file_list(lay_name):
#     directory_path = lay_name
#     files = os.listdir(directory_path)
# # create a new file for writing
#     list_file = "files.txt"
#     try:
#         os.remove(list_file)
#     except OSError:
#         pass
            
    
#     with open("files.txt", "w") as file:
#         # write each file name to the file
#         for f in files:
#             if f.endswith(".tif"):
#                 file.write(directory_path+"/reprojected/"+f + "\n")

### 3. Build vrt



In [10]:
#https://gdal.org/programs/gdal_translate.html
#cmdoption-gdal_translate-ovr


    

# def makecog(): 
#     print('Making COG..')
#     # cmd = f'''
#     # gdal_translate output.vrt cog.tif -of COG -co NUM_THREADS=ALL_CPUS -co COMPRESS=LZW -co BIGTIFF=YES
#     # '''
#     cmd = f'''
#     gdal_translate output.vrt cog2.tif -ot Float6 -of COG -r average 
#     -co COMPRESS=LZW -co BIGTIFF=YES -co TILING_SCHEME=GoogleMapsCompatible
#     '''

#     !{cmd}
#     print('COG Complete!')
    
    
# def makecog_rio(): 
#     print('Making COG..')

#     cmd = f'''
#     rio cogeo create output.vrt vrt_cog.tif --cog-profile lzw --config CHECK_DISK_FREE_SPACE=FALSE --overview-resampling average --allow-intermediate-compression'''

#     !{cmd}
#     print('COG Complete!')
    
# def make_geotiff():
#     print('Making Geotiff') 
#     cmd = f'gdal_translate output.vrt gtiff.tif -ot Float64 -of GTiff -r average -co COMPRESS=LZW' 
#     !{cmd}
#     print('Geotiff Complete!') 
# def ul(lay_name):
#     print('Uploading Layer...')
#     cmd = f'gsutil cp cog.tif gs://live_data_layers/rasters/{lay_name}.tif'
#     !{cmd}
#     print('Layer upload complete!') 

In [93]:
#https://gdal.org/programs/gdal_translate.html
#cmdoption-gdal_translate-ovr

def makevrt(lay_name):
    directory_path = lay_name+'/*.tif'
    print('Making VRT...')
    cmd = f'gdalbuildvrt  output.vrt {directory_path}'
    !{cmd}
    print('VRT Complete!')
    


    
    

    
# def make_geotiff():
#     print('Making Geotiff') 
#     cmd = f'gdal_translate output.vrt gtiff.tif -ot Float64 -of GTiff -r average -co COMPRESS=LZW' 
#     !{cmd}
#     print('Geotiff Complete!') 
def ul(lay_name):
    print('Uploading Layer...')
    cmd = f'gsutil cp {lay_name}_cog.tif gs://live_data_layers/rasters/{lay_name}.tif'
    !{cmd}
    print('Layer upload complete!') 

In [37]:
# def makecog(): 
#     print('Making COG..')
#     # cmd = f'''
#     # gdal_translate output.vrt cog.tif -of COG -co NUM_THREADS=ALL_CPUS -co COMPRESS=LZW -co BIGTIFF=YES
#     # '''
#     cmd = f'''
#     gdal_translate output.vrt cog_gdal.tif \
#     -stats \
#     -ot Float64 -of COG \
#     -co COMPRESS=LZW -co BIGTIFF=YES \
#     -co TILED=YES
#     -co OVERVIEWS=ignore_existing
#     '''
#     #-co TILING_SCHEME=GoogleMapsCompatible 
#     # -r average -a_nodata -9999\
#     #'''

#     !{cmd}
#     print('COG Complete!')

In [14]:
# lay_name = 'Precipitation_mm' 
# dl(lay_name)
# makevrt(lay_name)

# cmd1 = '''
# gdal_translate output.vrt tmp.tif \
# -co TILED=YES -co COMPRESS=LZW -co BIGTIFF=YES
# '''
# #cmd2 = 'gdaladdo -clean tmp.tif' 
# cmd3 = 'gdaladdo -r average tmp.tif' 
# cmd4 = f'''gdal_translate tmp.tif {lay_name}_cog.tif \
# -co TILED=YES -co COMPRESS=LZW -co COPY_SRC_OVERVIEWS=YES \
# -co BIGTIFF=YES
# '''

If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Copying gs://swhm-image-exports/Precipitation_mm/Precipitation_mm.tif...
/ [1/1 files][347.9 KiB/347.9 KiB] 100% Done                                    
Operation completed over 1 objects/347.9 KiB.                                    
Making VRT...
0...10...20...30...40...50...60...70...80...90...100 - done.
VRT Complete!


In [20]:
# cmd = 'gdalwarp -t_srs SR-ORG:6627 output.vrt tmpwarped.tif'
# !{cmd}

ERROR 1: Translating source or target SRS failed:
SR-ORG:6627
Usage: gdalwarp [--help-general] [--formats]
    [-s_srs srs_def] [-t_srs srs_def] [-to "NAME=VALUE"]* [-vshift | -novshift]
    [[-s_coord_epoch epoch] | [-t_coord_epoch epoch]]
    [-order n | -tps | -rpc | -geoloc] [-et err_threshold]
    [-refine_gcps tolerance [minimum_gcps]]
    [-te xmin ymin xmax ymax] [-tr xres yres] [-tap] [-ts width height]
    [-ovr level|AUTO|AUTO-n|NONE] [-wo "NAME=VALUE"] [-ot Byte/Int16/...] [-wt Byte/Int16]
    [-srcnodata "value [value...]"] [-dstnodata "value [value...]"] -dstalpha
    [-r resampling_method] [-wm memory_in_mb] [-multi] [-q]
    [-cutline datasource] [-cl layer] [-cwhere expression]
    [-csql statement] [-cblend dist_in_pixels] [-crop_to_cutline]
    [-if format]* [-of format] [-co "NAME=VALUE"]* [-overwrite]
    [-nomd] [-cvmd meta_conflict_value] [-setci] [-oo NAME=VALUE]*
    [-doo NAME=VALUE]*
    srcfile* dstfile

Available resampling methods:
    near (default), bili

In [11]:
!{cmd1}

Input file size is 380, 363
0...10...20...30...40...50...60...70...80...90...100 - done.


In [12]:
!{cmd3}

0...10...20...30...40...50...60...70...80...90...100 - done.


In [15]:
!{cmd4}

Input file size is 380, 363
0...10...20...30...40...50...60...70...80...90...100 - done.


In [63]:
# def makecog_rio(): 
#     print('Making COG with rio..')

#     cmd = f'''
#     rio cogeo create output.vrt vrt_cog.tif \
#     --resampling average --add-mask \
#     --cog-profile lzw --web-optimized \
#     --config CHECK_DISK_FREE_SPACE=FALSE \
#     --allow-intermediate-compression \
#     --blocksize 256 
#     '''

#     !{cmd}
#     print('COG Complete!')

---

## 4. Wrapper Function

In [90]:
def convert_layer(lay_name): 

    #dl(lay_name)
    
    #Make a vrt of the downloaded images 
    makevrt(lay_name)
    
    #check projection 
    p = subprocess.run(["rio", "info", "output.vrt"], capture_output=True, text=True)
    raster_info = json.loads(p.stdout)

#reproject if needed 
    if (raster_info['crs']) != "EPSG:3857": 
        #reproject 
        print(f'reprojecting from {raster_info["crs"]}')
        warp_cmd = 'gdalwarp -t_srs EPSG:3857 -overwrite output.vrt tmp-reproj.tif \
         -co NUM_THREADS=5 -co TILED=YES -co COMPRESS=LZW -co BIGTIFF=YES --config CHECK_DISK_FREE_SPACE FALSE'
        !{warp_cmd}
        !{'mv tmp-reproj.tif tmp.tif'}

            #rebuild pyramids if needed 
        #check data type
        if (raster_info["dtype"] != 'uint8'): 
            print(f'rebuilding overviews') 
            !{'gdaladdo -r average tmp.tif'}
        
    
    if(raster_info["dtype"] == "uint8" and raster_info['crs'] == "EPSG:3857"):
        print('translating output.vrt') 
        translate_cmd = f'gdal_translate output.vrt {lay_name}_cog.tif \
        -co TILED=YES -co COMPRESS=LZW -co COPY_SRC_OVERVIEWS=YES \
        -co BIGTIFF=YES \
         -co NUM_THREADS=5 -co TILED=YES --config CHECK_DISK_FREE_SPACE FALSE'
        !{translate_cmd}
    else:
        print('translating tmp.tif') 
        translate_cmd = f'gdal_translate tmp.tif {lay_name}_cog.tif \
        -co TILED=YES -co COMPRESS=LZW -co COPY_SRC_OVERVIEWS=YES \
        -co BIGTIFF=YES \
         -co NUM_THREADS=5 -co TILED=YES --config CHECK_DISK_FREE_SPACE FALSE'
        !{translate_cmd}

        

           



In [91]:
convert_layer("Precipitation_mm")

Making VRT...
0...10...20...30...40...50...60...70...80...90...100 - done.
VRT Complete!
translating tmp.tif
Input file size is 320651, 306809
0...10...20...30...40...50...60...70...80...90...100 - done.


In [94]:
ul("Precipitation_mm")

Uploading Layer...
Copying file://Precipitation_mm_cog.tif [Content-Type=image/tiff]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/ [1 files][  2.2 GiB/  2.2 GiB]  621.6 KiB/s                                   
Operation completed over 1 objects/2.2 GiB.                                      
Layer upload complete!


In [71]:
# #         print('doubs') 
# #         !{'gdaladdo -r average tmp.tif'} 
# def make_cog(lay_name):
#     cog_command = f'''gdal_translate tmp.tif {lay_name}_cog.tif \
#     -co TILED=YES -co COMPRESS=LZW -co COPY_SRC_OVERVIEWS=YES \
#     -co BIGTIFF=YES
#     '''
#     !{cog_command}

In [73]:
# lay_name = "Precipitation_mm"

# convert_layer(lay_name)

Making VRT...
0...10...20...30...40...50...60...70...80...90...100 - done.
VRT Complete!


AttributeError: 'dict' object has no attribute 'dtype'

In [78]:
# p = subprocess.run(["rio", "info", "output.vrt"], capture_output=True, text=True)
# raster_info = json.loads(p.stdout)
# raster_info["dtype"]
# raster_info['crs']



In [63]:
# warp_cmd = 'gdalwarp -t_srs EPSG:3857 -overwrite output.vrt tmp-reproj2.tif \
#          -co NUM_THREADS=5 -co TILED=YES -co COMPRESS=LZW -co BIGTIFF=YES --config CHECK_DISK_FREE_SPACE FALSE'
# !{warp_cmd}

Creating output file that is 320651P x 306809L.
Processing output.vrt [1/1] : 0...10.^C


In [60]:
# raster = gdal.Open('output.vrt')
# band = raster.GetRasterBand(1)
# data_type = gdal.GetDataTypeName(band.DataType)
# data_type

'Byte'

In [52]:
# # Open the file and check type 
# raster = gdal.Open('tmp.tif')
# band = raster.GetRasterBand(1)
# data_type = gdal.GetDataTypeName(band.DataType)


In [58]:
# # Build command 
# lay_name = "Land_Cover"
# cog_command = f'''gdal_translate tmp.tif {lay_name}_cog2.tif \
# -co TILED=YES -co COMPRESS=LZW -co COPY_SRC_OVERVIEWS=YES \
# -co BIGTIFF=YES -co NUM_THREADS=5 --config GDALSetCacheMax64 2'''
# cog_command
# !{cog_command}

Input file size is 320651, 306809
0...10...20...30...40...50...60...70...80...90...100 - done.


In [48]:
# lay_name = "Land_Cover"
# raster = gdal.Open('tmp.tif')
# band = raster.GetRasterBand(1)
# data_type = gdal.GetDataTypeName(band.DataType)
# print(data_type)

# #check projection 
# p = subprocess.run(["rio", "info", "tmp.tif"], capture_output=True, text=True)
# raster_info = json.loads(p.stdout)

# if (raster_info['crs']) != "EPSG:3857": 
#     #reproject 
#     print(f'reprojecting from {raster_info["crs"]}')
#     warp_cmd = 'gdalwarp -t_srs EPSG:3857 -overwrite  tmp.tif tmp-reproj.tif \
#     -co TILED=YES -co COMPRESS=LZW  -co BIGTIFF=YES -co NUM_THREADS=4'
#     !{warp_cmd}
#     #rename it and move on
#     !{"mv tmp-reproj.tif tmp.tif"}

# if data_type =="Double": 
#     print('doubs') 
#     !{'gdaladdo -r average tmp.tif'} 



Byte
reprojecting from EPSG:26910
Creating output file that is 320651P x 306809L.
Processing tmp.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.


In [66]:
# !{'gdaladdo -r average Land_Cover_cog.tif'} 

0Warning 1: Land_Cover_cog.tif: Adding new overviews invalidates the LAYOUT=IFDS_BEFORE_DATA property
...10...20...30...40...50.^C


# Upload Layer

## Get list of objects in data bucket

In [14]:
BUCKET_NAME = 'swhm-image-exports'
blobsout = list_blobs_with_prefix(BUCKET_NAME,'')

In [15]:
df = pd.DataFrame(blobsout, columns=['file_path'])#.iloc[1:]
#df['folder_name'] = df['file_path'].str.split(BUCKET_NAME, 1,expand = True)
df['gdal_path'] = df['file_path'].str.replace('gs://', '/vsigs/') 
df

,file_path,gdal_path
0,Flow_Duration_Index/Flow_Duration_Index0000000...,Flow_Duration_Index/Flow_Duration_Index0000000...
1,Flow_Duration_Index/Flow_Duration_Index0000000...,Flow_Duration_Index/Flow_Duration_Index0000000...
2,Flow_Duration_Index/Flow_Duration_Index0000000...,Flow_Duration_Index/Flow_Duration_Index0000000...
3,Flow_Duration_Index/Flow_Duration_Index0000000...,Flow_Duration_Index/Flow_Duration_Index0000000...
4,Flow_Duration_Index/Flow_Duration_Index0000000...,Flow_Duration_Index/Flow_Duration_Index0000000...
...,...,...
80,tss/tss0000023296-0000023296.tif,tss/tss0000023296-0000023296.tif
81,zinc/zinc0000000000-0000000000.tif,zinc/zinc0000000000-0000000000.tif
82,zinc/zinc0000000000-0000023296.tif,zinc/zinc0000000000-0000023296.tif
83,zinc/zinc0000023296-0000000000.tif,zinc/zinc0000023296-0000000000.tif


In [16]:
lay_names= df['file_path'].str.split('/', 0).str[0]#.str.replace('/','',regex=False)
df['layer_name'] = lay_names.str.split('/',1).str[0]
df
#lay_names

/var/folders/yn/15903z7124l3th5fm7wg0lgw0000gn/T/ipykernel_4889/3619429793.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  lay_names= df['file_path'].str.split('/', 0).str[0]#.str.replace('/','',regex=False)
/var/folders/yn/15903z7124l3th5fm7wg0lgw0000gn/T/ipykernel_4889/3619429793.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df['layer_name'] = lay_names.str.split('/',1).str[0]


,file_path,gdal_path,layer_name
0,Flow_Duration_Index/Flow_Duration_Index0000000...,Flow_Duration_Index/Flow_Duration_Index0000000...,Flow_Duration_Index
1,Flow_Duration_Index/Flow_Duration_Index0000000...,Flow_Duration_Index/Flow_Duration_Index0000000...,Flow_Duration_Index
2,Flow_Duration_Index/Flow_Duration_Index0000000...,Flow_Duration_Index/Flow_Duration_Index0000000...,Flow_Duration_Index
3,Flow_Duration_Index/Flow_Duration_Index0000000...,Flow_Duration_Index/Flow_Duration_Index0000000...,Flow_Duration_Index
4,Flow_Duration_Index/Flow_Duration_Index0000000...,Flow_Duration_Index/Flow_Duration_Index0000000...,Flow_Duration_Index
...,...,...,...
80,tss/tss0000023296-0000023296.tif,tss/tss0000023296-0000023296.tif,tss
81,zinc/zinc0000000000-0000000000.tif,zinc/zinc0000000000-0000000000.tif,zinc
82,zinc/zinc0000000000-0000023296.tif,zinc/zinc0000000000-0000023296.tif,zinc
83,zinc/zinc0000023296-0000000000.tif,zinc/zinc0000023296-0000000000.tif,zinc


In [18]:
lay_names = df["layer_name"].unique()
lay_names

array(['Flow_Duration_Index', 'Land_Cover', 'Land_Use', 'Slope', 'Soils',
       'copper', 'p', 'tkn', 'tss', 'zinc'], dtype=object)

In [27]:
filepath = 'Land_Cover/Land_Cover0000000000-0000000000.tif'
# Open the file:
raster = gdal.Open(filepath)
band = raster.GetRasterBand(1)
data_type = gdal.GetDataTypeName(band.DataType)

if data_type == 'Byte': 
    print('its a byte')


its a byte


In [20]:
dl(lay_names[1])

If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Copying gs://swhm-image-exports/Land_Cover/Land_Cover0000000000-0000000000.tif...
Copying gs://swhm-image-exports/Land_Cover/Land_Cover0000000000-0000065536.tif...
Copying gs://swhm-image-exports/Land_Cover/Land_Cover0000000000-0000131072.tif...
Copying gs://swhm-image-exports/Land_Cover/Land_Cover0000000000-0000196608.tif...
Copying gs://swhm-image-exports/Land_Cover/Land_Cover0000065536-0000065536.tif...
Copying gs://swhm-image-exports/Land_Cover/Land_Cover0000065536-0000000000.tif...
Copying gs://swhm-image-exports/Land_Cover/Land_Cover0000065536-0000131072.tif...
Copying gs://swhm-image-exports/Land_Cover/Land_Cover0000000000-0000262144.

In [ ]:
run_pipeline('Runoff_mm')

## Loop through layer names

Use this to run the pipeline for all layers in a list

In [ ]:
for lay_name in lay_names:
    run_pipeline(lay_name) 
    